# 11wk-1: `data_collator`

최규빈  
2024-11-22

<a href="https://colab.research.google.com/github/guebin/MP2024/blob/main/posts/11wk-1.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" style="text-align: left"></a>

# 1. 강의영상

<https://youtu.be/playlist?list=PLQqh36zP38-yntkaNrZmlqWVX-ineTf4k&si=xZ1WZainJuXiHheC>

# 2. Imports

In [2]:
import os
os.environ["WANDB_MODE"] = "offline"

In [3]:
import pandas as pd
import numpy as np
import datasets 
import transformers
import torch
import torchvision
import torch.utils
import evaluate

/home/cgb3/anaconda3/envs/hf/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

# 3. `data_collator` 이해

## A. 외우세요 $(\star\star\star)$

`-` `data_collator`를 잘 설계하는 방법: `trainer_input`과 `model`이
주어졌을때 `data_collator`는 아래의 코드가 동작하도록 설계하면 된다.

``` python
trainer_input = ~~~
model = ~~~~ 
#---#
batch_maker = transformers.Trainer(
    model = model,
    data_collator = lambda x: x
) # 이 과정에서 model이 cuda로 감 
_batched_data = batch_maker.get_test_dataloader(trainer_input) # 이 과정에서 trainer_input이 cuda로 감
batched_data = list(_batched_data)
single_batch = batched_data[-1]
model.to("cpu") # 경우에 따라 생략해야할수도있음
model(**data_collator(single_batch))
```

`-` 위의 코드가 오류없이 실행되었다면 아래의 코드를 사용할 수 있다.

``` python
trainer = transformers.Trainer(
    model = model,
    data_collator = data_collator
)
trainer.predict(trainer_input)
```

> 이걸 어떻게 알았냐고요? 코드뜯어봤습니다.. $\to$ 숙제

> **Important**
>
> 코랩사용자의 경우 아래와 같이 wandb(Weights & Biases) 로그인을
> 요구하는 문제가 있습니다.
>
> ``` bash
> wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
> wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
> wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
> wandb: You can find your API key in your browser here: https://wandb.ai/authorize
> wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:
> ```
>
> 이를 해결하기 위해서는 아래의 코드를 코랩처음에 실행하면 됩니다.
>
> ``` python
> import os
> os.environ["WANDB_MODE"] = "offline"
> ```

> **Note**
>
> 주의: `trainer_input`의 type이 꼭 `Dataset` 일 필요는 없다..

## B. IMDB – 복습

ref:
<https://huggingface.co/docs/transformers/tasks/sequence_classification>

*1. 데이터준비: `"guebin/imdb-tiny"` $\to$ `trainer_input`*

In [4]:
imdb = datasets.load_dataset("guebin/imdb-tiny")
tokenizer = transformers.AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased") 
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)
tokenized_imdb = imdb.map(preprocess_function,batched=True)
trainer_input = tokenized_imdb['train']
trainer_input

Map: 100%|█████████████████████████████| 10/10 [00:00<00:00, 1694.39 examples/s]

*2. 모델준비: `"distilbert/distilbert-base-uncased"` $\to$`model`*

In [5]:
model = transformers.AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.

*3. 데이터콜렉터: `DataCollatorWithPadding()` $\to$ `data_collator`*

In [6]:
data_collator = transformers.DataCollatorWithPadding(tokenizer=tokenizer)
data_collator

------------------------------------------------------------------------

데이터콜렉터가 올바로 설정되었는지 체크하고, 적당한 `trainer`를 만들어

``` python
trainer.predict(trainer_input)
```

이 정상동작하는지 확인하라.

`(풀이)`

In [13]:
batch_maker = transformers.Trainer(
    model = model,
    data_collator = lambda x: x
) # 이 과정에서 model이 cuda로 감 
_batched_data = batch_maker.get_test_dataloader(trainer_input) # 이 과정에서 trainer_input이 cuda로 감
batched_data = list(_batched_data)
single_batch = batched_data[-1]
model.to("cpu") # 경우에 따라 생략해야할수도있음
model(**data_collator(single_batch))

-   잘 돌아갔음. (=여기에서 사용된 데이터콜렉터는 잘 설계된
    `data_collator` 라는 의미)

In [25]:
trainer = transformers.Trainer(
    model = model,
    data_collator = data_collator
)
out = trainer.predict(trainer_input)
out 

`#`

`-` 관찰1: `batched_data[-1]` 는 하나의배치(single_batch)를 의미함.
모델의 입력으로는 부적절한 형식임.

In [51]:
# batched_data[-1] -- 부적절해보이는 모델입력..

In [52]:
model(**batched_data[-1])

`-` 관찰2: `data_collator(batched_data[-1])` 역시
하나의배치(single_batch)를 의미함. 그런데 이것은 모델의 입력으로도
적절한 형식.

In [59]:
data_collator(batched_data[-1]) # 모델의 입력으로 매우 바람직해 보이는 형식임 

In [65]:
model.to("cpu")
model(**data_collator(batched_data[1]))

> **`data_collator` – 심화이해**
>
> 아래의 형식으로 정리된 배치화된 자료가 있다고 하자. (주의:
> `batched_data`는 항상 list비슷한 오브젝트이어야함)
>
> ``` python
> batched_data = [batch_1, batch_2, ...,batch_n]
> ```
>
> `data_collator` 는 각각의 `single_batch`, 즉 `batch_1`, `batch_2` 등을
> `model`이 처리가능한 형태로 “형식”을 맞춰주는 역할을 한다. 즉 아래가
> 실행되도록 만들어주는 역할을 한다.
>
> ``` python
> model(**data_collator(batch_1))
> ```

> **`trainer`와 `model`의 자료처리과정 비교**
>
> ***#. `model`의 자료처리과정***
>
> -코드: `model.forward(model_input)`
>
> -처리과정: `model_input`에 정리된 입력을 단순히 `model.forward()`
> 함수가 처리.
>
> ***#. `trainer`의 자료처리과정***
>
> -코드: `trainer.predict(trainer_input)`
>
> -처리과정: 배치화 $\to$ 데이터콜렉팅 $\to$ 추론의 3단계를 거친다.
>
> 1.  `trainer_input`을 배치(batch)로 나눈다.
> 2.  각 배치(=`single_batch`)를 `data_collator`를 통해 형식을 맞춘다.
> 3.  형식이 조정된 데이터를 `model.forward`의 입력으로 전달한다.
>
> -슈도코드:
>
> ``` python
> ## 이 코드는.. 
> trainer.predict(trainer_input)
>
> ## 대략 아래의 느낌으로 해석하면 된다.. (동일X. 결과정리, GPU처리 등 세부로직이 더 있음)
> batched_data = some_function(trainer_input)
> for single_batch in batched_data:
>     collated_data = data_collator(single_batch)
>     model(**collated_data)
> ```

> **`trainer.predict()` 의 분해**
>
> `trainer.predict()`의 동작은 개념적으로 (1) 배치화 (2) 데이터콜렝팅
> (3) 추론의 과정으로 분해할 수 있지만, 실제이러한 과정으로 코드를
> 정확하게 분리하는건 어렵다. (그리고 저 사이사이에는 다른 자잘한
> 과정들이 많다..) 하지만 이해를 위해서 코드조각을 억지로 분리해본다면
> 아래 3개의 코드조각으로 분리할 수 있을것이다.
>
> `1`. 배치화: `trainer_input` $\to$ `batched_data`
>
> ``` python
> batch_maker = transformers.Trainer(
>     model = model,
>     data_collator = lambda x: x
> )
> _batched_data = batch_maker.get_test_dataloader(trainer_input)
> batched_data = list(_batched_data)
> ```
>
> `2`. 데이터콜렉팅: `single_batch` $\to$ `collated_data`
>
> ``` python
> #for single_batch in batched_data:
>     collated_data = data_collator(single_batch)
> ```
>
> `3`. 추론: `collated_data` $\to$ `model_out`
>
> ``` python
> #for single_batch in batched_data:
>     #collated_data = data_collator(single_batch)
>     model_out = model(**collated_data)
> ```

## C. FOOD101 – 복습

ref:
<https://huggingface.co/docs/transformers/tasks/image_classification>

*1. 데이터준비: `"guebin/food101-tiny"` $\to$ `trainer_input`*

In [11]:
food = datasets.load_dataset("guebin/food101-tiny")
image_processor = transformers.AutoImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")
normalize = torchvision.transforms.Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
_transforms = torchvision.transforms.Compose([
    torchvision.transforms.RandomResizedCrop(size), 
    torchvision.transforms.ToTensor(), 
    normalize
])
def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]
    return examples
trainer_input = food['train'].with_transform(transforms)
trainer_input

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.

*2. 모델준비: `"google/vit-base-patch16-224-in21k"` $\to$`model`*

In [24]:
labels = food["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label
model = transformers.AutoModelForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.

*3. 데이터콜렉터: `DefaultDataCollator()` $\to$ `data_collator`*

In [26]:
data_collator = transformers.DefaultDataCollator()
data_collator

------------------------------------------------------------------------

데이터콜렉터가 올바로 설정되었는지 체크하고, 적당한 `trainer`를 만들어

``` python
trainer.predict(trainer_input)
```

이 정상동작하는지 확인하라.

`(풀이1)` – 실패

In [27]:
batch_maker = transformers.Trainer(
    model = model,
    data_collator = lambda x: x 
)
_batched_data = batch_maker.get_test_dataloader(trainer_input)
batched_data = list(_batched_data)
single_batch = batched_data[-1]
model(**data_collator(single_batch))

`-` 왜 실패했지?? (예전에는 분명히 되었던 것 같은뎅..)

> **Note**
>
> **<에러메시지의 해석>**
>
> `-` 아래가 동작하지 않음.
>
> ``` python
> batched_data = list(_batched_data)
> ```
>
> `-` 그 이유는 아래가 동작하지 않기 때문임.
>
> ``` python
> next(dataloader_iter)
> ```
>
> `-` …(생략)…
>
> `-` 최종적으로는 아래가 동작하지 않기 때문에 생긴 문제였음. (그런데
> 이건 `.with_transform()`에 있는 코드인데?)
>
> ``` python
> examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
> ```
>
> `-` 결국
>
> ``` python
> [_transforms(img.convert("RGB")) for img in examples["image"]]
> ```
>
> 를 실행하는 시점에서 `examples["image"]`가 없었다는 의미.

> 눈치: `with_transform`이 지금 실행되는거였어?

`-` 왜 이런일이 생기지?

`-` 배치화를 하는 코드

``` python
_batched_data = batch_maker.get_test_dataloader(trainer_input)
```

에서 아래의 column_names:

-   `pixel_values`
-   `head_mask`
-   `labels`
-   `output_attentions`
-   `output_hidden_states`
-   `interpolate_pos_encoding`
-   `return_dict`

를 제외하고는 모두 트레이너(`batch_maker = trainer`)가 강제로 제거하는
로직이 있음.[1]

`-` `image`라는 column_name은 위에 해당되지 않으므로 제거됨.

`-` 그리고 `image` 칼럼이 제거된 이후에 `with_transform` 이 나중에
실행되면서 (지연실행) 문제가 발생.

> 이걸 어떻게 알았냐고요? 코드뜯어봤습니다.. $\to$ 숙제

> **중간정리**
>
> `trainer.predict()` 은 (1) 배치화 (2) 데이터콜렉팅 (3) 추론의 과정을
> 거친다. 그리고 배치화와 데이터콜렉팅 사이에 “싱글배치”를 만드는 과정이
> 있다.
>
> -   세부사항1: 그런데 “**배치화**”단계에서 `model.forward()`의
>     입력으로 사용되지 않는 columns는 지워지는 내부로직이 존재한다.
> -   세부사항2: `trainer_input`에 걸려있는 `.with_transform()`은
>     “**배치화**”이후 싱글배치가 만들어지는 과정에서 실행된다.
>
> 따라서 `.with_transform()` 에서 특정컬럼의 변화시키는 동작이 약속된
> 경우, 그 컬럼이 **배치화**의 단계에서 자동제거되어 코드가 돌아가지
> 않을 수 있는 위험성이 존재한다.

`(풀이2)` – `image`를 `return_dict` 로 위장.. // 완전 테크니컬한 풀이

`-` 현재상황: `food['train']`에 `.with_transform(transforms)`을
걸어두고(?) `trainer_input`을 만든상황

`-` 문제: `trainer.predict()` 내부동작에서 `.with_transform(transform)`
이 실현될때

[1] 왜 이런 로직이 있을까? 이런 로직이 없다면 model의 args를 강제로
외우고 있어야 하니까..

In [272]:
transforms??

Signature: transforms(examples)
Docstring: <no docstring>
Source:   
def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]
    return examples
File:      /tmp/ipykernel_706133/1515420127.py
Type:      function

이 내용이 실행되어야하는데, `image`는 model의 입력으로 유하하지 않은
키라서 트레이너가 이미 제거한 상태임.

`-` 전략: 제거가 안되게 막아보자..

In [44]:
#model.forward?

In [49]:
#trainer_input = food['train'].with_transform(transforms)
trainer_input2 = trainer_input.rename_columns({'image':'return_dict'})
trainer_input2

In [51]:
def transforms2(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["return_dict"]]
    del examples["return_dict"]
    return examples

In [55]:
trainer_input3 = trainer_input2.with_transform(transforms2)
trainer_input3

In [56]:
batch_maker = transformers.Trainer(
    model = model,
    data_collator = lambda x: x 
)
_batched_data = batch_maker.get_test_dataloader(trainer_input3)
batched_data = list(_batched_data)
single_batch = batched_data[-1]
model(**data_collator(single_batch))

In [59]:
trainer = transformers.Trainer(
    model = model,
    data_collator= data_collator
)
trainer.predict(trainer_input3)

`(풀이3)` – trainer_input 에 예약된 `with_transform`을 지연실행하지 않고
즉시 실행

In [61]:
trainer_input

In [69]:
trainer_input2 = [l for l in trainer_input]
#trainer_input2

In [71]:
batch_maker = transformers.Trainer(
    model = model,
    data_collator = lambda x: x 
)
_batched_data = batch_maker.get_test_dataloader(trainer_input2)
batched_data = list(_batched_data)
single_batch = batched_data[-1]
model(**data_collator(single_batch))

In [72]:
trainer = transformers.Trainer(
    model = model,
    data_collator= data_collator
)
trainer.predict(trainer_input2)

`(풀이4)` – 트레이너가 가진 “사용하지 않는 column을 제거하는 기능”을
`False` 시킴..

In [79]:
trainer_input

In [80]:
batch_maker = transformers.Trainer(
    model = model,
    data_collator = lambda x: x,
    args = transformers.TrainingArguments(
        output_dir="asdf",
        remove_unused_columns=False
    )
)
_batched_data = batch_maker.get_test_dataloader(trainer_input)
batched_data = list(_batched_data)
single_batch = batched_data[-1]
model(**data_collator(single_batch))

In [82]:
trainer = transformers.Trainer(
    model = model,
    data_collator= data_collator,
    args = transformers.TrainingArguments(
        output_dir="asdf",
        remove_unused_columns=False
    )    
)
trainer.predict(trainer_input)

`#`

`(풀이5)` – 트레이너가 가진 “사용하지 않는 column을 제거하는 기능”을
`False` 시킬꺼면, `batch_maker`를 고려할 필요도 없이 아래와 같이 바로
`single_batch`를 얻을 수 있음.

*풀이4: 실제로 trainer가 싱글배치를 얻는 과정과 유사하게 얻는 방법*

In [89]:
batch_maker = transformers.Trainer(
    model = model,
    data_collator = lambda x: x,
    args = transformers.TrainingArguments(
        output_dir= "asdf", # 아무거나 써야함. 
        remove_unused_columns= False # 이 부분이 포인트!!
    )        
)
_batched_data = batch_maker.get_test_dataloader(trainer_input)
batched_data = list(_batched_data)
single_batch = batched_data[0]
# single_batch = [
#     {'label':int, 'pixel_values': 3d-tsr},
#     {'label':int, 'pixel_values': 3d-tsr},
#     {'label':int, 'pixel_values': 3d-tsr},
#     {'label':int, 'pixel_values': 3d-tsr},
#     {'label':int, 'pixel_values': 3d-tsr},
#     {'label':int, 'pixel_values': 3d-tsr},
#     {'label':int, 'pixel_values': 3d-tsr},
#     {'label':int, 'pixel_values': 3d-tsr},
# ]    

> 형식관찰: `single_batch`는 `[Dict, Dict, Dict, .... Dict]` 꼴임을
> 주목하라.

*풀이5: 형식관찰에 힌트를 얻어 무식하게 얻은 싱글배치*

In [113]:
single_batch = [
    trainer_input[0],
    trainer_input[1],
    trainer_input[2],
    trainer_input[3],
    trainer_input[4],
    trainer_input[5],
    trainer_input[6],
    trainer_input[7],
]

*아무튼 풀이5 스타일로 싱글배치를 얻었다면? 이후의 코드는 동일*

In [114]:
model.to("cpu")
model(**data_collator(single_batch));

In [99]:
trainer = transformers.Trainer(
    model = model,
    data_collator= data_collator,
    args = transformers.TrainingArguments(
        output_dir="asdf",
        remove_unused_columns=False
    )    
)
trainer.predict(trainer_input)

*참고1: 아래의 방식으로 싱글배치를 얻을 수 없음. – 이유?
지연실행때문에..*

In [115]:
#single_batch = trainer_input.to_list()[:8]

*참고2: 아래의 방식으로도 싱글배치를 얻을 수 없음.*

In [116]:
#single_batch = trainer_input[:8]

*이유?*

In [108]:
trainer_input[:2] == [trainer_input[0],trainer_input[1]]

## D. FOOD101 – DefaultDataCollator 구현

*1. 데이터준비: `"guebin/food101-tiny"` $\to$ `trainer_input`*

In [117]:
food = datasets.load_dataset("guebin/food101-tiny")
image_processor = transformers.AutoImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")
normalize = torchvision.transforms.Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
_transforms = torchvision.transforms.Compose([
    torchvision.transforms.RandomResizedCrop(size), 
    torchvision.transforms.ToTensor(), 
    normalize
])
def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]
    return examples
trainer_input = food['train'].with_transform(transforms)
trainer_input

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.

*2. 모델준비: `"google/vit-base-patch16-224-in21k"` $\to$`model`*

In [118]:
labels = food["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label
model = transformers.AutoModelForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.

*3. 데이터콜렉터: `collate_fn` 직접설계*

In [55]:
# data_collator = transformers.DefaultDataCollator()
# data_collator

In [119]:
def collate_fn(single_batch):
    pass

`DefaultDataCollator()` 와 동일한 역할을 하는 `collate_fn`을 설계하라.
이를 이용하여 적당한 `trainer`를 만들어

``` python
trainer.predict(trainer_input)
```

이 정상동작하는지 확인하라.

`(풀이)`

In [120]:
trainer_input

In [132]:
# batch_maker = transformers.Trainer(
#     model= model,
#     data_collator= lambda x: x,
#     args = transformers.TrainingArguments(
#         output_dir="asdf",
#         remove_unused_columns=False
#     )
# )
# _batched_data = batch_maker.get_eval_dataloader(trainer_input)
# batched_data = list(_batched_data)
# single_batch = batched_data[-1]
#---#
single_batch = [trainer_input[-2],trainer_input[-1]]
single_batch

In [153]:
def collate_fn(single_batch):
    #single_batch = [Dict,Dict] 
    #Dict = {'label': 6, 'pixel_values': [3, 224, 224]-tensor
    collated_data = dict()
    collated_data['labels'] = torch.tensor([dct['label'] for dct in single_batch])    
    collated_data['pixel_values'] = torch.stack([dct['pixel_values'] for dct in single_batch])
    return collated_data

In [142]:
model.to("cpu")
model(**collate_fn(single_batch))

In [145]:
trainer = transformers.Trainer(
    model=model,
    data_collator=collate_fn,
    args=transformers.TrainingArguments(
        output_dir="asdf",
        remove_unused_columns=False
    )
)
trainer.predict(trainer_input)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).

***** Running Prediction *****
  Num examples = 10
  Batch size = 8

------------------------------------------------------------------------

## E. IMDB – DataCollatorWithPadding 구현

ref:
<https://huggingface.co/docs/transformers/tasks/sequence_classification>

*1. 데이터준비: `"guebin/imdb-tiny"` $\to$ `trainer_input`*

In [3]:
imdb = datasets.load_dataset("guebin/imdb-tiny")
tokenizer = transformers.AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)
tokenized_imdb = imdb.map(preprocess_function,batched=True)
trainer_input = tokenized_imdb['train']

*2. 모델준비: `"distilbert/distilbert-base-uncased"` $\to$`model`*

In [4]:
model = transformers.AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.

*3. 데이터콜렉터: `collate_fn` 직접설계*

In [6]:
# data_collator = transformers.DataCollatorWithPadding(tokenizer=tokenizer)
# data_collator

In [7]:
def collate_fn(single_batch):
    pass

------------------------------------------------------------------------

`DataCollatorWithPadding()` 와 동일한 역할을 하는 `collate_fn`을
설계하라. 이를 이용하여 적당한 `trainer`를 만들어

``` python
trainer.predict(trainer_input)
```

이 정상동작하는지 확인하라.

`(풀이)`

In [8]:
trainer_input

In [60]:
batch_maker = transformers.Trainer(
    model= model,
    data_collator= lambda x: x,
)
_batched_data = batch_maker.get_eval_dataloader(trainer_input)
batched_data = list(_batched_data)
single_batch = batched_data[-1]

In [61]:
labels = torch.tensor([dct['label'] for dct in single_batch])
labels

In [62]:
input_ids = torch.nn.utils.rnn.pad_sequence([torch.tensor(dct['input_ids']) for dct in single_batch]).t()
input_ids

In [63]:
attention_mask = torch.nn.utils.rnn.pad_sequence([torch.tensor(dct['attention_mask']) for dct in single_batch]).t()
attention_mask

In [64]:
# single_batch = [Dict, Dict]
# Dict = {
#     'label': int 
#     'input_ids': 1d-list 
#     'attention_mask': 1d-list 
# }
def collate_fn(single_batch):
    collated_data = dict()
    collated_data['input_ids'] = torch.nn.utils.rnn.pad_sequence([torch.tensor(dct['input_ids']) for dct in single_batch]).t()    
    collated_data['attention_mask'] = torch.nn.utils.rnn.pad_sequence([torch.tensor(dct['attention_mask']) for dct in single_batch]).t()
    collated_data['labels'] = torch.tensor([dct['label'] for dct in single_batch])
    return collated_data

In [65]:
collate_fn(single_batch)

In [66]:
model.to("cpu")
model(**collate_fn(single_batch))

In [78]:
trainer = transformers.Trainer(
    model=model,
    data_collator=collate_fn,
)
trainer.predict(trainer_input)

# 4. 연습 – `sms_spam`

In [3]:
model = transformers.AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2
)
tokenizer = transformers.AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")
spam = datasets.load_dataset('guebin/spam-tiny')
spam

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.

In [5]:
spam

## A. 방법1: 고정패딩, `collate_fn`

In [202]:
def m_trans(example_batch):
    # example_batch = {'sms':[xxx,xxxx,...], 'label':[yyy,yyyy] 
    # example_batch = spam['train'][:8]
    out = tokenizer(example_batch['sms'],padding=True,truncation=True)
    return out 

In [203]:
spam2 = spam.map(m_trans,batched=True,batch_size=8)
spam2

Map: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 1538.07 examples/s]

In [204]:
spam2.set_format("pt")
#spam2['train']['input_ids'] -- list of tensor with length 10 

In [205]:
spam2['train'][8:]['input_ids'] # 2d-tensor 

In [206]:
spam2['train'][7:]['input_ids'] # list of 1d-tensor 

In [207]:
trainer_input = spam2['train'].remove_columns(['sms']).rename_columns({'label':'labels'})
trainer_input

In [208]:
batch_maker = transformers.Trainer(
    model= model,
    data_collator=lambda x:x
) 
_batched_data = batch_maker.get_eval_dataloader(trainer_input)
batched_data = list(_batched_data)
single_batch = batched_data[-1]
single_batch

In [209]:
torch.stack([single_batch[0]['labels'],single_batch[1]['labels']])

In [210]:
def collate_fn(single_batch):
    out = dict()
    out['labels'] = torch.stack([dct['labels'] for dct in single_batch])
    out['input_ids'] = torch.stack([dct['input_ids'] for dct in single_batch])
    out['attention_mask'] = torch.stack([dct['attention_mask'] for dct in single_batch])
    return out 

In [211]:
model(**collate_fn(single_batch))

In [212]:
trainer = transformers.Trainer(
    model= model,
    data_collator=collate_fn
)
trainer.predict(trainer_input)

In [213]:
trainer = transformers.Trainer(
    model=model,
    data_collator=collate_fn,
    train_dataset=trainer_input,
    args = transformers.TrainingArguments(
        output_dir="asdf",
        remove_unused_columns=False
    )
)
trainer.train()

## B. 방법2: 고정패딩, DefaultDataCollator

In [195]:
def m_trans(example_batch):
    # example_batch = {'sms':[xxx,xxxx,...], 'label':[yyy,yyyy] 
    # example_batch = spam['train'][:8]
    out = tokenizer(example_batch['sms'],padding=True,truncation=True)
    return out 
spam2 = spam.map(m_trans,batched=True,batch_size=8)
spam2.set_format("pt")
trainer_input = spam2['train'].remove_columns(['sms']).rename_columns({'label':'labels'})

Map: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 1685.61 examples/s]

In [196]:
batch_maker = transformers.Trainer(
    model= model,
    data_collator=lambda x:x
) 
_batched_data = batch_maker.get_eval_dataloader(trainer_input)
batched_data = list(_batched_data)
single_batch = batched_data[-1]
single_batch

In [197]:
# def collate_fn(single_batch):
#     out = dict()
#     out['labels'] = torch.stack([dct['labels'] for dct in single_batch])
#     out['input_ids'] = torch.stack([dct['input_ids'] for dct in single_batch])
#     out['attention_mask'] = torch.stack([dct['attention_mask'] for dct in single_batch])
#     return out 
data_collator = transformers.DefaultDataCollator()

In [198]:
model(**data_collator(single_batch))

In [199]:
trainer = transformers.Trainer(
    model= model,
    data_collator=data_collator
)
trainer.predict(trainer_input)

In [201]:
trainer = transformers.Trainer(
    model=model,
    data_collator=data_collator,
    train_dataset=trainer_input,
    args = transformers.TrainingArguments(
        output_dir="asdf",
        remove_unused_columns=False
    )
)
trainer.train()

## C. 방법3: 동적패딩, `DataCollatorWithPadding`

In [187]:
spam

In [188]:
def w_trans(examples):
    # examples = spam['train'][:8] = {'sms': [xxx,xxxx,...], 'label':[yyy,yyyy,...]
    out = tokenizer(examples['sms'],truncation=True)
    out['labels'] = torch.tensor(examples['label'])
    return out 

In [189]:
trainer_input = spam.with_transform(w_trans)['train']
trainer_input

In [190]:
batch_maker = transformers.Trainer(
    model = model,
    data_collator = lambda x: x,
    args = transformers.TrainingArguments(
        output_dir="asdf",
        remove_unused_columns=False
    )
)
single_batch = next(iter(batch_maker.get_eval_dataloader(trainer_input)))
#sigle_batch

In [191]:
data_collator = transformers.DataCollatorWithPadding(tokenizer)
model.to("cpu")
model(**data_collator(single_batch))

In [192]:
trainer = transformers.Trainer(
    model = model,
    data_collator = data_collator,
    args = transformers.TrainingArguments(
        output_dir="asdf",
        remove_unused_columns=False
    )
)
trainer.predict(trainer_input)

In [194]:
trainer = transformers.Trainer(
    model=model,
    data_collator=data_collator,
    train_dataset=trainer_input,
    args = transformers.TrainingArguments(
        output_dir="asdf",
        remove_unused_columns=False
    )
)
trainer.train()

## D. 방법4: 동적패딩, 전처리X $(\star)$

In [166]:
trainer_input = spam['train']
trainer_input

In [167]:
single_batch = [trainer_input[-2],trainer_input[-1]]
single_batch

In [181]:
def collate_fn(single_batch):
    out = tokenizer(
        [dct['sms'] for dct in single_batch],
        padding=True,
        truncation=True,
        return_tensors="pt",
    )
    out['labels'] = torch.tensor([dct['label'] for dct in single_batch])
    return out 

In [182]:
model.to("cpu")
model(**collate_fn(single_batch))

In [184]:
trainer = transformers.Trainer(
    model=model,
    data_collator=collate_fn,
    args = transformers.TrainingArguments(
        output_dir="asdf",
        remove_unused_columns=False
    )
)
trainer.predict(trainer_input)

In [186]:
trainer = transformers.Trainer(
    model=model,
    data_collator=collate_fn,
    train_dataset=trainer_input,
    args = transformers.TrainingArguments(
        output_dir="asdf",
        remove_unused_columns=False
    )
)
trainer.train()

------------------------------------------------------------------------

# A1. 공지

> **강의시간 이슈**
>
> 안녕하세요, 제가 촬영하고 강의시간을 살펴보니 원래 강의시간보다 약
> 20분정도 초과되었습니다. (3시간 분량인데 3시간20분 소요됨) 죄송합니다.
> 이후의 강의에서 이를 반영하여 조금 강의시간을 줄여서
> 올리도록하겠습니다.

> **깊은복사 얕은복사**
>
> 아래의 코드
>
> ``` python
> lst = [1,2,3]
> lst2 = lst 
> lst2.append(4)
> ```
>
> 를 실행하였을 경우 `lst`와 `lst2`에 동일한 값이 저장되는 현상에 대한
> 설명은
>
> > <https://guebin.github.io/PP2023/posts/2023-06-21-13wk-1.html>
>
> 에 있으니 관심있으신 학생들은 참고하시기 바랍니다. (이 수업에서는 저
> 내용을 몰라도 학점받는데 영향없습니다)